In [1]:
import numpy as np
import pandas as pd
import pandasql
import sqlite3

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2) #2 titik perpuluhan


In [2]:
date = 20220630
RATE_1 = 20220630

In [3]:
#PRODUCT_HIERARCHY
Product_Hierarchy = pd.read_csv (r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\02_User_Defined\06_Product_Hierarchy\Product_Hierarchy.txt", sep = "\t", header = 0, low_memory = False)
Product_Hierarchy.columns = Product_Hierarchy.columns.str.replace(" ", "_")


In [4]:
#PRODUCT_TYPE_CODES
Product_Type_Codes = pd.read_csv(r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\02_User_Defined\09_Product_Type_Codes\Product_Type_Codes.txt", sep = "\t", header = 0, low_memory = False)
Product_Type_Codes.columns = Product_Type_Codes.columns.str.replace(" ", "_")

In [5]:
#Loans
location_loan = r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\01_MIB_Loans\\"+str(date)+"\\MIB_Loans"+str(date)+".txt"
loan = pd.read_csv(location_loan, sep = "\t", header = 0, low_memory = False)
loan.columns = loan.columns.str.replace(" ", "_")

In [117]:
loan1 = loan[['M_Account_No','M_Cus_No','M_Source_Prod_Type_Cd','M_Product_Hierarchy_Cd','M_Bnm_Balance_SUM']]



PRD = Product_Hierarchy[['Product_Hierarchy_Cd','Source_Prod_Type_Dsc']]

PRD.Product_Hierarchy_Cd = PRD.Product_Hierarchy_Cd.astype(str)
PRD.Product_Hierarchy_Cd = PRD.Product_Hierarchy_Cd.str.upper()
PRD.Product_Hierarchy_Cd = PRD.Product_Hierarchy_Cd.str.strip()

loan1.M_Product_Hierarchy_Cd = loan1.M_Product_Hierarchy_Cd.astype(str)
loan1.M_Product_Hierarchy_Cd = loan1.M_Product_Hierarchy_Cd.str.upper()
loan1.M_Product_Hierarchy_Cd = loan1.M_Product_Hierarchy_Cd.str.strip()

loan_merge = loan1.merge(PRD.rename(columns={'Product_Hierarchy_Cd': 'M_Product_Hierarchy_Cd',\
                                            'Source_Prod_Type_Dsc':'Source_Prod_Type_Dsc(Prod)'}), on = 'M_Product_Hierarchy_Cd', how = 'left')

PRD_2 = Product_Hierarchy[['Source_Prod_Type_Cd', 'Source_Prod_Type_Dsc']]

PRD_2.Source_Prod_Type_Cd = PRD_2.Source_Prod_Type_Cd.astype(str)
PRD_2.Source_Prod_Type_Cd = PRD_2.Source_Prod_Type_Cd.str.upper()
PRD_2.Source_Prod_Type_Cd = PRD_2.Source_Prod_Type_Cd.str.strip()

loan1.M_Source_Prod_Type_Cd = loan1.M_Source_Prod_Type_Cd.astype(str)
loan1.M_Source_Prod_Type_Cd = loan1.M_Source_Prod_Type_Cd.str.upper()
loan1.M_Source_Prod_Type_Cd = loan1.M_Source_Prod_Type_Cd.str.strip()

loan_merge1 = loan_merge.merge(PRD_2.rename(columns={'Source_Prod_Type_Cd': 'M_Source_Prod_Type_Cd',\
                                                    'Source_Prod_Type_Dsc':'Source_Prod_Type_Dsc(Source)'}), on = ["M_Source_Prod_Type_Cd"], how="left")


loan_prove = loan_merge1.iloc[np.where(loan_merge1['Source_Prod_Type_Dsc(Prod)']!=loan_merge1['Source_Prod_Type_Dsc(Source)'])]

loan_prove_nodup = loan_merge1.iloc[np.where(loan_merge1['Source_Prod_Type_Dsc(Prod)']!=loan_merge1['Source_Prod_Type_Dsc(Source)'])].drop_duplicates()

D:\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [118]:
loan.shape, loan_merge.shape, loan_merge1.shape, loan_prove.shape, loan_prove_nodup.shape

((757369, 40), (757369, 6), (757369, 7), (11312, 7), (11312, 7))

In [18]:
#OD
location_OD = r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\05_MIB_OD\\"+str(date)+"\\MIB_OD"+str(date)+".txt"
OD = pd.read_csv(location_OD, sep = "\t", header = 0)
OD.columns = OD.columns.str.replace(" ", "_")

D:\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (20) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [119]:
OD1 = OD[['Account_No','Cus_No','Source_Prod_Type_Cd','Product_Hierarchy_Cd','M_Bnm_Balance']]



PRD = Product_Hierarchy[['Product_Hierarchy_Cd','Source_Prod_Type_Dsc']]

PRD.Product_Hierarchy_Cd = PRD.Product_Hierarchy_Cd.astype(str)
PRD.Product_Hierarchy_Cd = PRD.Product_Hierarchy_Cd.str.upper()
PRD.Product_Hierarchy_Cd = PRD.Product_Hierarchy_Cd.str.strip()

OD1.Product_Hierarchy_Cd = OD1.Product_Hierarchy_Cd.astype(str)
OD1.Product_Hierarchy_Cd = OD1.Product_Hierarchy_Cd.str.upper()
OD1.Product_Hierarchy_Cd = OD1.Product_Hierarchy_Cd.str.strip()

OD_merge = OD1.merge(PRD.rename(columns={\
                                            'Source_Prod_Type_Dsc':'Source_Prod_Type_Dsc(Prod)'}), on = 'Product_Hierarchy_Cd', how = 'left')

PRD_2 = Product_Hierarchy[['Source_Prod_Type_Cd', 'Source_Prod_Type_Dsc']]

PRD_2.Source_Prod_Type_Cd = PRD_2.Source_Prod_Type_Cd.astype(str)
PRD_2.Source_Prod_Type_Cd = PRD_2.Source_Prod_Type_Cd.str.upper()
PRD_2.Source_Prod_Type_Cd = PRD_2.Source_Prod_Type_Cd.str.strip()

OD1.Source_Prod_Type_Cd = OD1.Source_Prod_Type_Cd.astype(str)
OD1.Source_Prod_Type_Cd = OD1.Source_Prod_Type_Cd.str.upper()
OD1.Source_Prod_Type_Cd = OD1.Source_Prod_Type_Cd.str.strip()

OD_merge1 = OD_merge.merge(PRD_2.rename(columns={\
                                                    'Source_Prod_Type_Dsc':'Source_Prod_Type_Dsc(Source)'}), on = ["Source_Prod_Type_Cd"], how="left")


OD_prove = OD_merge1.iloc[np.where(OD_merge1['Source_Prod_Type_Dsc(Prod)']!=OD_merge1['Source_Prod_Type_Dsc(Source)'])]

OD_prove_nodup = OD_merge1.iloc[np.where(OD_merge1['Source_Prod_Type_Dsc(Prod)']!=OD_merge1['Source_Prod_Type_Dsc(Source)'])].drop_duplicates()


In [120]:
OD.shape, OD_merge.shape, OD_merge1.shape, OD_prove.shape, OD_prove_nodup.shape

((34437, 48), (34437, 6), (135965, 7), (135962, 7), (36256, 7))

In [30]:
location_inv = r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\06_MIB_Investment\\"+str(date)+"\\MIB_INVESTMENT"+str(date)+".txt"
inv = pd.read_csv(location_inv, sep = "\t", header = 0)
inv.columns = inv.columns.str.replace(" ", "_")

In [121]:
inv1 = inv[['M_Account_No','M_Cus_No','M_Source_Prod_Type_Cd','M_Product_Hierarchy_Cd','M_Bnm_Balance']]



PRD = Product_Hierarchy[['Product_Hierarchy_Cd','Source_Prod_Type_Dsc']]

PRD.Product_Hierarchy_Cd = PRD.Product_Hierarchy_Cd.astype(str)
PRD.Product_Hierarchy_Cd = PRD.Product_Hierarchy_Cd.str.upper()
PRD.Product_Hierarchy_Cd = PRD.Product_Hierarchy_Cd.str.strip()

inv1.M_Product_Hierarchy_Cd = inv1.M_Product_Hierarchy_Cd.astype(str)
inv1.M_Product_Hierarchy_Cd = inv1.M_Product_Hierarchy_Cd.str.upper()
inv1.M_Product_Hierarchy_Cd = inv1.M_Product_Hierarchy_Cd.str.strip()

inv_merge = inv1.merge(PRD.rename(columns={'Product_Hierarchy_Cd': 'M_Product_Hierarchy_Cd',\
                                            'Source_Prod_Type_Dsc':'Source_Prod_Type_Dsc(Prod)'}), on = 'M_Product_Hierarchy_Cd', how = 'left')

PRD_2 = Product_Hierarchy[['Source_Prod_Type_Cd', 'Source_Prod_Type_Dsc']]

PRD_2.Source_Prod_Type_Cd = PRD_2.Source_Prod_Type_Cd.astype(str)
PRD_2.Source_Prod_Type_Cd = PRD_2.Source_Prod_Type_Cd.str.upper()
PRD_2.Source_Prod_Type_Cd = PRD_2.Source_Prod_Type_Cd.str.strip()

inv1.M_Source_Prod_Type_Cd = inv1.M_Source_Prod_Type_Cd.astype(str)
inv1.M_Source_Prod_Type_Cd = inv1.M_Source_Prod_Type_Cd.str.upper()
inv1.M_Source_Prod_Type_Cd = inv1.M_Source_Prod_Type_Cd.str.strip()

inv_merge1 = inv_merge.merge(PRD_2.rename(columns={'Source_Prod_Type_Cd': 'M_Source_Prod_Type_Cd',\
                                                    'Source_Prod_Type_Dsc':'Source_Prod_Type_Dsc(Source)'}), on = ["M_Source_Prod_Type_Cd"], how="left")


inv_prove = inv_merge1.iloc[np.where(inv_merge1['Source_Prod_Type_Dsc(Prod)']!=inv_merge1['Source_Prod_Type_Dsc(Source)'])]

inv_prove_nodup = inv_merge1.iloc[np.where(inv_merge1['Source_Prod_Type_Dsc(Prod)']!=inv_merge1['Source_Prod_Type_Dsc(Source)'])].drop_duplicates()

In [122]:
inv.shape, inv_merge.shape, inv_merge1.shape, inv_prove.shape, inv_prove_nodup.shape

((10947, 34), (10947, 6), (20602, 7), (204, 7), (92, 7))

In [36]:
location_trade = r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\07_MIB_Trade\\"+str(date)+"\\MIB_TRADEBILLS"+str(date)+".txt"
trade = pd.read_csv(location_trade, sep = "\t", header = 0)
trade.columns = trade.columns.str.replace(" ", "_")

In [37]:
trade.head(1)

,Account_No,Cus_No,Amrt_Term,Amrt_Term_Mult,As_Of_Date,Branch_Code,Common_Coa_Id,Issue_Date,Record_Count,Source_Prod_Type_Cd,Interest_Income,Non_Interest_Income,Out_Of_Pocket_Exp,Fees_Earned,Commission_Earned,Interest_In_Suspense,Specific_Provision,Economic_Sector_Cd,Funded_Flg,Interest_Income_Myr,Non_Interest_Income_Myr,Out_Of_Pocket_Exp_Myr,Fees_Earned_Myr,Commission_Earned_Myr,Interest_In_Suspense_Myr,Specific_Provision_Myr,Total_Activity_Cost,Maintenance_Cost,M_Bnm_Balance,Entity_Cd,M_Sub_Market_Segment_Cd,Product_Hierarchy_Cd,M_Financing_Concept,Cur_Gross_Rate,Avg_Book_Bal,Avg_Book_Bal_Myr,Funds_Charge,Net_Int_Income,Funds_Charge_Myr,Net_Int_Income_Myr,Ca_Account_No,Gl_Account_Id
0,99140SLU5437246,4560825,8096,D,30-JUN-2022,1415.0,22000,28-FEB-2007,1,IBG-PERFM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,41001C,N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,37.8,20000.0,MIB,5.0,IBG-PERFM,19,0.0,20000.0,20000.0,NaN,NaN,NaN,NaN,014150421451,389210117411


In [123]:
trade1 = trade[['Account_No','Cus_No','Source_Prod_Type_Cd','Product_Hierarchy_Cd','M_Bnm_Balance']]



PRD = Product_Type_Codes[['Code','Description']]

PRD.Code = PRD.Code.astype(str)
PRD.Code = PRD.Code.str.upper()
PRD.Code = PRD.Code.str.strip()

trade1.Product_Hierarchy_Cd = trade1.Product_Hierarchy_Cd.astype(str)
trade1.Product_Hierarchy_Cd = trade1.Product_Hierarchy_Cd.str.upper()
trade1.Product_Hierarchy_Cd = trade1.Product_Hierarchy_Cd.str.strip()

trade_merge = trade1.merge(PRD.rename(columns={'Code':'Product_Hierarchy_Cd',\
                                            'Description':'Source_Prod_Type_Dsc(Prod)'}), on = 'Product_Hierarchy_Cd', how = 'left')

PRD_2 = Product_Type_Codes[['Code', 'Description']]

PRD_2.Code = PRD_2.Code.astype(str)
PRD_2.Code = PRD_2.Code.str.upper()
PRD_2.Code = PRD_2.Code.str.strip()

trade_merge.Source_Prod_Type_Cd = trade_merge.Source_Prod_Type_Cd.astype(str)
trade_merge.Source_Prod_Type_Cd = trade_merge.Source_Prod_Type_Cd.str.upper()
trade_merge.Source_Prod_Type_Cd = trade_merge.Source_Prod_Type_Cd.str.strip()

trade_merge1 = trade_merge.merge(PRD_2.rename(columns={'Code':'Source_Prod_Type_Cd',\
                                                    'Description':'Source_Prod_Type_Dsc(Source)'}), on = ["Source_Prod_Type_Cd"], how="left")


trade_prove = trade_merge1.iloc[np.where(trade_merge1['Source_Prod_Type_Dsc(Prod)']!=trade_merge1['Source_Prod_Type_Dsc(Source)'])]

trade_prove_nodup = trade_merge1.iloc[np.where(trade_merge1['Source_Prod_Type_Dsc(Prod)']!=trade_merge1['Source_Prod_Type_Dsc(Source)'])].drop_duplicates()


In [124]:
trade.shape, trade_merge.shape, trade_merge1.shape, trade_prove.shape, trade_prove_nodup.shape

((24960, 42), (26492, 6), (31088, 7), (3064, 7), (1532, 7))

In [55]:
location_fcl = r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\08_MIB_FCL\\"+str(date)+"\\MIB_FCL"+str(date)+".txt"
fcl = pd.read_csv(location_fcl, sep = "\t", header = 0)
fcl.columns = fcl.columns.str.replace(" ", "_")

In [56]:
fcl.head(1)

,M_Account_No,M_Cus_No,Common_Coa_Id,As_Of_Date,Amrt_Term,Amrt_Term_Mult,Branch_Code,Cur_Gross_Rate,Issue_Date,Record_Count,M_Date_Aa_Approved,M_Acct_Status_Cd,M_Approved_Limit,M_Approved_Limit_Myr,M_Source_Prod_Type_Cd,M_Mths_Installment_In_Arrears,M_Special_Mention_Acct_Flg,M_Bnm_Balance,M_Interest_Income,M_Penalty_Charges,M_Economic_Sector_Cd,M_Full_Writeoff_Flg,M_Reschedule_Flg,M_Restructure_Flg,M_Npl_Flg,M_New_Npl_Flg,M_Profit_On_Cost,M_Entity_Cd,M_Sub_Market_Segment_Cd,M_Product_Hierarchy_Cd,M_Financing_Concept,Avg_Book_Bal,Avg_Book_Bal_Myr,M_Interest_Income_Myr,M_Interest_Code,M_Aa_Number,Gl_Account_Id,M_Date_Final_Released,M_Date_First_Released,M_Rescheduled_Date,M_Restructured_Date,M_Npl_Date
0,764454800004,46578,12000,30-JUN-2022,240,M,6445,0.0,23-DEC-2014,0,20-OCT-2014,1,722505.0,3.86e+06,W14,0,N,2.29e+06,1733.72,0.0,310,N,NaN,NaN,N,N,1733.72,MIB,3,W14,A04,426191.95,2.32e+06,9270.31,43,64454 201445952,120301210114,24-DEC-2014,24-DEC-2014,01-JAN-1900,01-JAN-1900,01-JAN-1900


In [125]:
fcl1 = fcl[['M_Account_No','M_Cus_No','M_Source_Prod_Type_Cd','M_Product_Hierarchy_Cd','M_Bnm_Balance']]



PRD = Product_Hierarchy[['Product_Hierarchy_Cd','Source_Prod_Type_Dsc']]

PRD.Product_Hierarchy_Cd = PRD.Product_Hierarchy_Cd.astype(str)
PRD.Product_Hierarchy_Cd = PRD.Product_Hierarchy_Cd.str.upper()
PRD.Product_Hierarchy_Cd = PRD.Product_Hierarchy_Cd.str.strip()

fcl1.M_Product_Hierarchy_Cd = fcl1.M_Product_Hierarchy_Cd.astype(str)
fcl1.M_Product_Hierarchy_Cd = fcl1.M_Product_Hierarchy_Cd.str.upper()
fcl1.M_Product_Hierarchy_Cd = fcl1.M_Product_Hierarchy_Cd.str.strip()

fcl_merge = fcl1.merge(PRD.rename(columns={'Product_Hierarchy_Cd': 'M_Product_Hierarchy_Cd',\
                                            'Source_Prod_Type_Dsc':'Source_Prod_Type_Dsc(Prod)'}), on = 'M_Product_Hierarchy_Cd', how = 'left')

PRD_2 = Product_Hierarchy[['Source_Prod_Type_Cd', 'Source_Prod_Type_Dsc']]

PRD_2.Source_Prod_Type_Cd = PRD_2.Source_Prod_Type_Cd.astype(str)
PRD_2.Source_Prod_Type_Cd = PRD_2.Source_Prod_Type_Cd.str.upper()
PRD_2.Source_Prod_Type_Cd = PRD_2.Source_Prod_Type_Cd.str.strip()

fcl1.M_Source_Prod_Type_Cd = fcl1.M_Source_Prod_Type_Cd.astype(str)
fcl1.M_Source_Prod_Type_Cd = fcl1.M_Source_Prod_Type_Cd.str.upper()
fcl1.M_Source_Prod_Type_Cd = fcl1.M_Source_Prod_Type_Cd.str.strip()

fcl_merge1 = fcl_merge.merge(PRD_2.rename(columns={'Source_Prod_Type_Cd': 'M_Source_Prod_Type_Cd',\
                                                    'Source_Prod_Type_Dsc':'Source_Prod_Type_Dsc(Source)'}), on = ["M_Source_Prod_Type_Cd"], how="left")


fcl_prove = fcl_merge1.iloc[np.where(fcl_merge1['Source_Prod_Type_Dsc(Prod)']!=fcl_merge1['Source_Prod_Type_Dsc(Source)'])]

fcl_prove_nodup = fcl_merge1.iloc[np.where(fcl_merge1['Source_Prod_Type_Dsc(Prod)']!=fcl_merge1['Source_Prod_Type_Dsc(Source)'])].drop_duplicates()


In [126]:
fcl.shape, fcl_merge.shape, fcl_merge1.shape, fcl_prove.shape, fcl_prove_nodup.shape

((535, 42), (535, 6), (535, 7), (0, 7), (0, 7))

In [62]:
location_HP = r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\09_MIB_HP\\"+str(RATE_1)+"\MIB_HP"+str(RATE_1)+".txt"
location_HPFS = r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\09_MIB_HP\\"+str(RATE_1)+"\MIB_HPFS"+str(RATE_1)+".txt"
location_HPBD = r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\09_MIB_HP\\"+str(RATE_1)+"\MIB_HPBD"+str(RATE_1)+".txt"
location_applicant = r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\09_MIB_HP\\"+str(RATE_1)+"\LOS_HPS_APPLICANT_DETAILS"+str(RATE_1)+".txt"
location_application = r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\09_MIB_HP\\"+str(RATE_1)+"\LOS_HPS_APPLICATION_DETAILS"+str(RATE_1)+".txt"

HP = pd.read_csv(location_HP, sep = "\t", header = 0)
HPFS = pd.read_csv(location_HPFS, sep = "\t", header = 0)
HPBD = pd.read_csv(location_HPBD, sep = "\t", header = 0)
applicant = pd.read_csv(location_applicant,sep = "\t", header = 0)
application = pd.read_csv(location_application,sep = "\t", header = 0)


HP.columns = HP.columns.str.replace(" ", "_")
HPFS.columns = HPFS.columns.str.replace(" ", "_")
HPBD.columns = HPBD.columns.str.replace(" ", "_")
applicant.columns = applicant.columns.str.replace(" ", "_")
application.columns = application.columns.str.replace(" ", "_")

In [73]:
HP1 = HP[['Account_No','Cus_No','Source_Prod_Type_Cd','Product_Hierarchy_Cd','Bnm_Balance']] 
HPFS1=HPFS[['Account_No','Cus_No','Source_Prod_Type_Cd','Product_Hierarchy_Cd','Bnm_Balance']]
HPBD1 = HPBD[['Account_No','Cus_No','Source_Prod_Type_Cd','Product_Hierarchy_Cd','Bnm_Balance']]

HP1.columns = HPFS1.columns = HPBD1.columns

aitab = pd.concat([HP1, HPFS1, HPBD1])

In [127]:
aitab1 = aitab[['Account_No','Cus_No','Source_Prod_Type_Cd','Product_Hierarchy_Cd','Bnm_Balance']]



PRD = Product_Hierarchy[['Product_Hierarchy_Cd','Source_Prod_Type_Dsc']]

PRD.Product_Hierarchy_Cd = PRD.Product_Hierarchy_Cd.astype(str)
PRD.Product_Hierarchy_Cd = PRD.Product_Hierarchy_Cd.str.upper()
PRD.Product_Hierarchy_Cd = PRD.Product_Hierarchy_Cd.str.strip()

aitab1.Product_Hierarchy_Cd = aitab1.Product_Hierarchy_Cd.astype(str)
aitab1.Product_Hierarchy_Cd = aitab1.Product_Hierarchy_Cd.str.upper()
aitab1.Product_Hierarchy_Cd = aitab1.Product_Hierarchy_Cd.str.strip()

aitab_merge = aitab1.merge(PRD.rename(columns={'Product_Hierarchy_Cd': 'Product_Hierarchy_Cd',\
                                            'Source_Prod_Type_Dsc':'Source_Prod_Type_Dsc(Prod)'}), on = 'Product_Hierarchy_Cd', how = 'left')

PRD_2 = Product_Hierarchy[['Source_Prod_Type_Cd', 'Source_Prod_Type_Dsc']]

PRD_2.Source_Prod_Type_Cd = PRD_2.Source_Prod_Type_Cd.astype(str)
PRD_2.Source_Prod_Type_Cd = PRD_2.Source_Prod_Type_Cd.str.upper()
PRD_2.Source_Prod_Type_Cd = PRD_2.Source_Prod_Type_Cd.str.strip()

aitab1.Source_Prod_Type_Cd = aitab1.Source_Prod_Type_Cd.astype(str)
aitab1.Source_Prod_Type_Cd = aitab1.Source_Prod_Type_Cd.str.upper()
aitab1.Source_Prod_Type_Cd = aitab1.Source_Prod_Type_Cd.str.strip()

aitab_merge1 = aitab_merge.merge(PRD_2.rename(columns={'Source_Prod_Type_Cd': 'Source_Prod_Type_Cd',\
                                                    'Source_Prod_Type_Dsc':'Source_Prod_Type_Dsc(Source)'}), on = ["Source_Prod_Type_Cd"], how="left")


aitab_prove = aitab_merge1.iloc[np.where(aitab_merge1['Source_Prod_Type_Dsc(Prod)']!=aitab_merge1['Source_Prod_Type_Dsc(Source)'])]

aitab_prove_nodup = aitab_merge1.iloc[np.where(aitab_merge1['Source_Prod_Type_Dsc(Prod)']!=aitab_merge1['Source_Prod_Type_Dsc(Source)'])].drop_duplicates()

In [128]:
aitab.shape, aitab_merge.shape, aitab_merge1.shape, aitab_prove.shape, aitab_prove_nodup.shape

((1012599, 5), (1012599, 6), (1013540, 7), (0, 7), (0, 7))

In [85]:
location_CC = r"T:\MIB Risk Management\Credit Risk Analytics - Historical\04_Infocube\99_Data_Source\01_Datawarehouse\01_Credit_Risk_Reporting\13_MIB_Credit_Cards\\"+str(date)+"\\MIB_CC"+str(date)+".txt"
CC = pd.read_csv(location_CC, sep = "\t", header = 0, error_bad_lines = False)
CC.columns = CC.columns.str.replace(" ", "_")

In [86]:
CC.head(1)

,As_Of_Date,M_Account_No,M_Cus_No,M_Acct_Open_Date,M_Entity_Cd,Branch_Code,M_Card_Block_Cd,M_Card_Status_Cd,M_Bnm_Balance,M_Expiry_Date,M_Grace_Days,M_New_Npl_Flg,M_Npl_Date,M_Past_Due_Days,M_Principal_Flg,M_Source_Prod_Type_Cd,Record_Count,M_Sub_Market_Segment_Cd,M_Product_Hierarchy_Cd,M_Mths_In_Arrears,M_Cust_Credit_Limit,M_Financing_Type_Cd,M_Doubtful_Flag,Cycle_Day_Of_Month,M_Behavioural_Score
0,30-JUN-2022,4046110000034681,27865883,18-FEB-2022,MIB,14002,P,1,0.0,28-FEB-2027,22,N,01-JAN-1900,2,Y,19,0,1.0,19,0,0.0,I,N,31,0


In [133]:
CC1 = CC[['M_Account_No','M_Cus_No','M_Source_Prod_Type_Cd','M_Product_Hierarchy_Cd','M_Bnm_Balance']]



PRD = Product_Hierarchy[['Product_Hierarchy_Cd','Source_Prod_Type_Dsc']]

PRD.Product_Hierarchy_Cd = PRD.Product_Hierarchy_Cd.astype(str)
PRD.Product_Hierarchy_Cd = PRD.Product_Hierarchy_Cd.str.upper()
PRD.Product_Hierarchy_Cd = PRD.Product_Hierarchy_Cd.str.strip()

CC1['M_Product_Hierarchy_Cd'] = str(0) + CC1['M_Product_Hierarchy_Cd'].astype(str)
CC1.M_Product_Hierarchy_Cd = CC1.M_Product_Hierarchy_Cd.astype(str)
CC1.M_Product_Hierarchy_Cd = CC1.M_Product_Hierarchy_Cd.str.upper()
CC1.M_Product_Hierarchy_Cd = CC1.M_Product_Hierarchy_Cd.str.strip()

CC_merge = CC1.merge(PRD.rename(columns={'Product_Hierarchy_Cd': 'M_Product_Hierarchy_Cd',\
                                            'Source_Prod_Type_Dsc':'Source_Prod_Type_Dsc(Prod)'}), on = 'M_Product_Hierarchy_Cd', how = 'left')

PRD_2 = Product_Hierarchy[['Source_Prod_Type_Cd', 'Source_Prod_Type_Dsc']]

PRD_2.Source_Prod_Type_Cd = PRD_2.Source_Prod_Type_Cd.astype(str)
PRD_2.Source_Prod_Type_Cd = PRD_2.Source_Prod_Type_Cd.str.upper()
PRD_2.Source_Prod_Type_Cd = PRD_2.Source_Prod_Type_Cd.str.strip()

CC_merge['M_Source_Prod_Type_Cd'] = str(0) + CC_merge['M_Source_Prod_Type_Cd'].astype(str)
CC_merge.M_Source_Prod_Type_Cd = CC_merge.M_Source_Prod_Type_Cd.astype(str)
CC_merge.M_Source_Prod_Type_Cd = CC_merge.M_Source_Prod_Type_Cd.str.upper()
CC_merge.M_Source_Prod_Type_Cd = CC_merge.M_Source_Prod_Type_Cd.str.strip()

CC_merge1 = CC_merge.merge(PRD_2.rename(columns={'Source_Prod_Type_Cd': 'M_Source_Prod_Type_Cd',\
                                                    'Source_Prod_Type_Dsc':'Source_Prod_Type_Dsc(Source)'}), on = ["M_Source_Prod_Type_Cd"], how="left")


CC_prove = CC_merge1.iloc[np.where(CC_merge1['M_Product_Hierarchy_Cd']!=CC_merge1['M_Source_Prod_Type_Cd'])]

CC_prove_nodup = CC_merge1.iloc[np.where(CC_merge1['M_Product_Hierarchy_Cd']!=CC_merge1['M_Source_Prod_Type_Cd'])].drop_duplicates()

D:\anaconda3\lib\site-packages\pandas\core\generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
D:\anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


In [134]:
CC.shape, CC_merge.shape, CC_merge1.shape, CC_prove.shape, CC_prove_nodup.shape

((1068500, 25), (1068500, 6), (1068500, 7), (0, 7), (0, 7))

In [116]:
ProdCCProof = Product_Hierarchy.iloc[np.where(Product_Hierarchy.Source_Prod_Type_Cd.isin(['020','20','051','51',20,51]))]
ProdCCProof

,Product_Hierarchy_Cd,Product_Hierarchy_Dsc,Source_Prod_Type_Cd,Source_Prod_Type_Dsc,Data_Source,Level_0,Level_1,Level_2,Level_3,Level_4,Level_5,Level_6,Level_7,Level_8,Instrument_Type_Cd,Last_Update_By,Last_Update_Dt,Last_Update_Dt:_Year,Last_Update_Dt:_Quarter,Last_Update_Dt:_Month,Last_Update_Dt:_Day
771,CCA_20_12000,All Product -> On Balance Sheet -> Assets -> Loans And Advances -> OD -> Personal -> Other Personal Loan,20,CONTRACT PLUS,3.0,All Product,On Balance Sheet,Assets,Loans And Advances,OD,Personal,Other Personal Loan,Other Personal Loan,Other Personal Loan,211.0,SYSOFSA,25-JUL-2011,2011.0,Q3,Jul,25.0
835,ICA_20_12000,All Product -> On Balance Sheet -> Assets -> Loans And Advances -> OD -> Personal -> Other Personal Loan,20,CONTRACT PLUS,3.0,All Product,On Balance Sheet,Assets,Loans And Advances,OD,Personal,Other Personal Loan,Other Personal Loan,Other Personal Loan,211.0,SYSOFSA,25-JUL-2011,2011.0,Q3,Jul,25.0
899,NET_20_12000,All Product -> On Balance Sheet -> Assets -> Loans And Advances -> OD -> Personal -> Other Personal Loan,20,CONTRACT PLUS,3.0,All Product,On Balance Sheet,Assets,Loans And Advances,OD,Personal,Other Personal Loan,Other Personal Loan,Other Personal Loan,211.0,SYSOFSA,25-JUL-2011,2011.0,Q3,Jul,25.0
963,PCA_20_12000,All Product -> On Balance Sheet -> Assets -> Loans And Advances -> OD -> Personal -> Other Personal Loan,20,CONTRACT PLUS,3.0,All Product,On Balance Sheet,Assets,Loans And Advances,OD,Personal,Other Personal Loan,Other Personal Loan,Other Personal Loan,211.0,SYSOFSA,25-JUL-2011,2011.0,Q3,Jul,25.0
1027,PMA_20_12000,All Product -> On Balance Sheet -> Assets -> Loans And Advances -> OD -> Personal -> Other Personal Loan,20,CONTRACT PLUS,3.0,All Product,On Balance Sheet,Assets,Loans And Advances,OD,Personal,Other Personal Loan,Other Personal Loan,Other Personal Loan,211.0,SYSOFSA,25-JUL-2011,2011.0,Q3,Jul,25.0
1935,NET_51_12000,All Product -> On Balance Sheet -> Assets -> Loans And Advances -> OD -> Government Aided Schemes -> CGC,51,FGS-RFSB,3.0,All Product,On Balance Sheet,Assets,Loans And Advances,OD,Government Aided Schemes,CGC,CGC,CGC,211.0,SYSOFSA,27-JUN-2016,2016.0,Q2,Jun,27.0
1936,CCA_51_12000,All Product -> On Balance Sheet -> Assets -> Loans And Advances -> OD -> Government Aided Schemes -> CGC,51,FGS-RFSB,3.0,All Product,On Balance Sheet,Assets,Loans And Advances,OD,Government Aided Schemes,CGC,CGC,CGC,211.0,SYSOFSA,27-JUN-2016,2016.0,Q2,Jun,27.0
1937,ICA_51_12000,All Product -> On Balance Sheet -> Assets -> Loans And Advances -> OD -> Government Aided Schemes -> CGC,51,FGS-RFSB,3.0,All Product,On Balance Sheet,Assets,Loans And Advances,OD,Government Aided Schemes,CGC,CGC,CGC,211.0,SYSOFSA,27-JUN-2016,2016.0,Q2,Jun,27.0
1938,PCA_51_12000,All Product -> On Balance Sheet -> Assets -> Loans And Advances -> OD -> Government Aided Schemes -> CGC,51,FGS-RFSB,3.0,All Product,On Balance Sheet,Assets,Loans And Advances,OD,Government Aided Schemes,CGC,CGC,CGC,211.0,SYSOFSA,27-JUN-2016,2016.0,Q2,Jun,27.0
1939,PMA_51_12000,All Product -> On Balance Sheet -> Assets -> Loans And Advances -> OD -> Government Aided Schemes -> CGC,51,FGS-RFSB,3.0,All Product,On Balance Sheet,Assets,Loans And Advances,OD,Government Aided Schemes,CGC,CGC,CGC,211.0,SYSOFSA,27-JUN-2016,2016.0,Q2,Jun,27.0


In [103]:
CC_prove.M_Source_Prod_Type_Cd.value_counts()

020    104
051     58
Name: M_Source_Prod_Type_Cd, dtype: int64

In [136]:
writer2 = pd.ExcelWriter(r'D:\Users\80005701\Desktop\Work\Kerja\Issue-(Prod & Source).xlsx',engine='xlsxwriter')

loan_prove_nodup.to_excel(writer2, sheet_name='loan', index = False)
OD_prove_nodup.to_excel(writer2, sheet_name='OD', index = False)
inv_prove_nodup.to_excel(writer2, sheet_name='inv', index = False)
fcl_prove_nodup.to_excel(writer2, sheet_name='fcl', index = False)
trade_prove_nodup.to_excel(writer2, sheet_name='trade', index = False)
aitab_prove_nodup.to_excel(writer2, sheet_name='aitab', index = False)
CC_prove_nodup.to_excel(writer2, sheet_name='CC', index = False)

writer2.save()